#### Idea

- Access spotify with authorization. Get one(all) of your playlists and tracks. 
- Get information/features of those tacks and cluster them together to find trends in your taste of music.
- Maybe sort by dates and see how your taste has evolved?
- ^I like this idea

---Damn I think I am taking my music too seriously

##### Roadmap
We don't have time, so lets get it working quickly. Following are the steps
- Understand how to interact with spotify api
- register your app
- get access to your playlists. Get songs from a playlist. This should conclude phase 1.
- get features for your songs. Save them in database? Plot them?
- Advanced: 
  - Run a clustering algorithm to find labels to define your music
  - Try to find related artists, go through their music and find similar?

In [1]:
import spotipy
sp = spotipy.Spotify()

results = sp.search(q='beethoven', limit=20)
for i, t in enumerate(results['tracks']['items']):
    print (' ', i, t['name'])

  0 Sonata No. 14 "Moonlight" in C-Sharp Minor", Op. 27 No. 2: I. Adagio sostenuto
  1 Horn Sonata in F Major, Op. 17 (version for flute and fortepiano): I. Allegro moderato
  2 Horn Sonata in F Major, Op. 17 (version for flute and fortepiano): III. Allegro moderato
  3 Horn Sonata in F Major, Op. 17 (version for flute and fortepiano): II. Poco adagio, quasi andante
  4 Flute Sonata in B-Flat Major, Anh. 4 (version for flute and fortepiano): I. Allegro moderato
  5 Flute Sonata in B-Flat Major, Anh. 4 (version for flute and fortepiano): II. Polonaise
  6 Flute Sonata in B-Flat Major, Anh. 4 (version for flute and fortepiano): III. Largo
  7 Flute Sonata in B-Flat Major, Anh. 4 (version for flute and fortepiano): IV. Allegretto molto con variazione
  8 Serenade in D Major, Op. 41 (version for flute and fortepiano): I. Entrata: Allegro
  9 Serenade in D Major, Op. 41 (version for flute and fortepiano): II. Tempo ordinario d'un menuetto
  10 Serenade in D Major, Op. 41 (version for flute 

In [2]:

# from flask import Flask, request, render_template
# from flask import Flask, request, session, g, redirect, url_for, abort, \
#      render_template, flash
# from werkzeug.utils import secure_filename
import sqlite3
import os
import requests
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json


### Basic Requests

In [3]:
#coldpaly - spotify:artist:4gzpq5DPGxSnKTe4SA8HAU
response = requests.get("https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU")
print (response)

<Response [200]>


In [4]:
jsonstr = response.content.decode()
jsonstr

'{\n  "external_urls" : {\n    "spotify" : "https://open.spotify.com/artist/4gzpq5DPGxSnKTe4SA8HAU"\n  },\n  "followers" : {\n    "href" : null,\n    "total" : 6316938\n  },\n  "genres" : [ "permanent wave", "pop", "pop christmas", "rock" ],\n  "href" : "https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU",\n  "id" : "4gzpq5DPGxSnKTe4SA8HAU",\n  "images" : [ {\n    "height" : 1000,\n    "url" : "https://i.scdn.co/image/31fa9aa639d539dc93ab9e3570c7c70a1b5c64f8",\n    "width" : 1000\n  }, {\n    "height" : 640,\n    "url" : "https://i.scdn.co/image/f700aaf6ebf1f0105d4eece756d924ab7efc3441",\n    "width" : 640\n  }, {\n    "height" : 200,\n    "url" : "https://i.scdn.co/image/32e13d2c37af4817d1cbf09742c1f51a328820c7",\n    "width" : 200\n  }, {\n    "height" : 64,\n    "url" : "https://i.scdn.co/image/3f31f2b7e890c2e59a9e4d67ea37f459e7036991",\n    "width" : 64\n  } ],\n  "name" : "Coldplay",\n  "popularity" : 91,\n  "type" : "artist",\n  "uri" : "spotify:artist:4gzpq5DPGxSnKTe4SA8H

In [5]:
jsloads = json.loads(jsonstr)

In [6]:
jsloads['name']

'Coldplay'

In [7]:
#coldpaly - spotify:artist:4gzpq5DPGxSnKTe4SA8HAU
# response = requests.get("https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU")
response = requests.get("https://api.spotify.com/v1/search?q='coldplay'&type=artist")
print (response)
jsonstr = response.content.decode()
jsloads = json.loads(jsonstr)


<Response [200]>


In [8]:
jsloads['artists'].keys()

dict_keys(['next', 'items', 'total', 'limit', 'previous', 'offset', 'href'])

In [9]:
jsloads['artists']['items'][2]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/58naUc4Fd9OoP6UrdrpgFO'},
 'followers': {'href': None, 'total': 72},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/58naUc4Fd9OoP6UrdrpgFO',
 'id': '58naUc4Fd9OoP6UrdrpgFO',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/7e544bb9b858905f95ed9aa96a186ceeca94f282',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/072dd1a6fcf580629efa78f55c4e76df6a509780',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/03ca05365ef52b26f844a359a27cd0f59cadd00c',
   'width': 64}],
 'name': 'ColdPlay Wu',
 'popularity': 0,
 'type': 'artist',
 'uri': 'spotify:artist:58naUc4Fd9OoP6UrdrpgFO'}

<hr />
<hr />


## Authorization
Phew, the following needs to be summarized for future dumb me. <br>
Basically, to authorize, you send the client id and redirect uri of your app to the spotify. This is done by accessing accounts.spotify.com/authorize?response_type=code?client_id=xyz. <br>
**Client_id** tells which app is asking for authorization. It prompts you to login, which is basically asking which account you want to access. If the login is successful, it redirects you to **redirect_uri**. <br>
This is the url/server which is listening. When you are redirected to this link, its path is appended with **?code =** after which spotify is giving you the authorization response, basically stating that yea, everything is okay<br>
You can then use this response to get the access token(which needs to be refreshed and stuff). <br>
Good workflow is here http://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#web-application-flow

In [18]:
import os 
#becuase by default, OAUTH2 requires https but we are at http.
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'


In [15]:
#Details of you 
client_id = "42b6ab36875d43128f6b750da5c0e987"
client_secret = "8c1dcb159aee4d69962e0b357f048eec"
redirect_uri = 'http://127.0.0.1:8080/callback/q'
#'http://localhost:5000/callback'
# 'http://localhost:8888/callback'
# http://127.0.0.1:8080/callback/q
auth_url = "https://accounts.spotify.com/authorize"
token_url = "https://accounts.spotify.com/api/token"

In [16]:
from requests_oauthlib import OAuth2Session

scopes = 'user-read-private user-read-email'
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri,
                          scope=scopes, state = "34fFs29kd09")
authorization_url, state = oauth.authorization_url(auth_url)
print ('Please go to %s and authorize access.' % authorization_url)
authorization_response =  input('Enter the URL you were redirected to')
### Note that the page might say This site can’t be reached and connection refused, 
### but you are only interested in url
### This last bit is important!

Please go to https://accounts.spotify.com/authorize?response_type=code&client_id=42b6ab36875d43128f6b750da5c0e987&redirect_uri=http%3A%2F%2F127.0.0.1%3A8080%2Fcallback%2Fq&scope=user-read-private+user-read-email&state=34fFs29kd09 and authorize access.
Enter the full callback URLhttp://127.0.0.1:8080/callback/q?code=AQCI1Ofs4pVTBu13O24QPvDY0cKJ-FSGKRNLDkaMrUpxUgYNZy0hBS7AcHS8k4lsostHSy_NdVRZD7cpv9M3i_nJAqq66Vvf7HgpbyyZfge6r4rMdpMln2vzTBEM9I0g-I0SGu_M6TcdyRwCn3UPZ0vGW8d7_6Zo6lDJZEfl9DEYQv6Dp6GYckNlRbqk_-vk2x9gf5dmGH7u0A1HnLbblRZfIMRbkWFiW5T4oCfT286Um50xRnDfnByW&state=34fFs29kd09


In [19]:
token = oauth.fetch_token(
        token_url,
        authorization_response=authorization_response,
        client_secret=client_secret)

In [20]:
token

{'access_token': 'BQATnSIQTt_x3_ChRyGPX5Qy90dqaP1CmJGe8cggFWQXjm38tiofxpLCO-x7xrwUUHbKKzNr4aGcQcDUt2Ht_r7RbXKl4onpjf2fFAKtUcPJlEthZew8Z3-ePQiBVQe70YVM8JKUFc1dYrFvv_boViPO0DdPJokoTUI',
 'expires_at': 1478158564.673078,
 'expires_in': 3600,
 'refresh_token': 'AQBgwE4uh38lOTkIErhvOO5B9UsdTbMa8KPER0NHHl927R_lN8rD8KYuqUAVqmbh4OgKLpyOKWUR7Bgjs65YNpputnp58mL6JHwCtm5jYUvI3aBBlj06TBD7Hw7qX_6Bq9E',
 'scope': ['user-read-email', 'user-read-private'],
 'token_type': 'Bearer'}

In [23]:
response = oauth.get("https://api.spotify.com/v1/me")
response

<Response [200]>

In [26]:
response.content.decode()

'{\n  "country" : "US",\n  "display_name" : "Chirag Modi",\n  "email" : "greatestchirag@gmail.com",\n  "external_urls" : {\n    "spotify" : "https://open.spotify.com/user/1189956848"\n  },\n  "followers" : {\n    "href" : null,\n    "total" : 28\n  },\n  "href" : "https://api.spotify.com/v1/users/1189956848",\n  "id" : "1189956848",\n  "images" : [ {\n    "height" : null,\n    "url" : "https://scontent.xx.fbcdn.net/v/t1.0-1/s200x200/944125_668663203159382_294146875_n.jpg?oh=ad7583c93bac7e7f245ff9c87b5e2f80&oe=588D0B72",\n    "width" : null\n  } ],\n  "product" : "premium",\n  "type" : "user",\n  "uri" : "spotify:user:1189956848"\n}'